#**Import Modules**

Selenium, Openpyxl, Files, Time

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2,
         "download_restrictions": 3}
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 2.8MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [141 kB]
Get:12 http://archive.ubuntu.com/ubuntu b

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [0]:
from selenium.common.exceptions import *

In [0]:
import time

In [0]:
import openpyxl

In [0]:
from google.colab import files

#**Mount Google Drive**

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [7]:
cd drive/My Drive/Colab Notebooks/Paper/Telecommunication

/content/drive/My Drive/Colab Notebooks/Paper/Telecommunication


In [8]:
ls

Crawler/  Data/  Paper-Keywords/


#**Fetch all Volume Links**

In [11]:
MAINURLFORMAT = "https://www.sciencedirect.com/journal/information-economics-and-policy/issues?page="
volUrls = []

for i in range(2):
    # access page
    print("Reading, " + MAINURLFORMAT + str(i + 1))
    wd.get(MAINURLFORMAT + str(i + 1))
    time.sleep(0.5)

    # toggle all dropdown buttons
    btns = wd.find_elements_by_xpath("//button[@aria-expanded]")
    for btn in btns:
        if "accordion" in btn.get_attribute("id"):
            while "true" not in btn.get_attribute("aria-expanded"):
                btn.click()
                time.sleep(0.5)

    # fetch volume urls
    a_tags = wd.find_elements_by_xpath("//a[@href]")
    for tag in a_tags:
        if "vol" in tag.get_attribute("href") and not tag.get_attribute("href") in volUrls:
            volUrls.append(tag.get_attribute("href"))

Reading, https://www.sciencedirect.com/journal/information-economics-and-policy/issues?page=1
Reading, https://www.sciencedirect.com/journal/information-economics-and-policy/issues?page=2


In [0]:
# check missing volume no.
tmps = []

for volurl in volUrls:
    tmp = volurl[volurl.index("vol") + 4:]
    tmp = tmp[:tmp.index("/")]
    if tmp not in tmps:
        tmps.append(tmp)

for i in range(49):
    if str(i + 1) not in tmps:
        print(i + 1)

#**Fetch and Write Data from each Volumes**

In [0]:
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "kwd"

In [0]:
TARGETPATH = "./Data/raw/International/"

In [15]:
 r = 2

 # write header
 ws.cell(row = 1, column = 1).value = 'title'
 ws.cell(row = 1, column = 2).value = 'journal'
 ws.cell(row = 1, column = 3).value = 'year'
 ws.cell(row = 1, column = 4).value = 'keywords'
 wb.create_sheet("auth")
 ws = wb["auth"]
 ws.cell(row = 1, column = 1).value = 'title'
 ws.cell(row = 1, column = 2).value = 'authors'
 ws = wb["kwd"]

 for volurl in volUrls:
    # access volume page
    wd.get(volurl)
    time.sleep(1)

    # fetch volume no. from url
    vol = volurl[volurl.index("vol") + 4:]
    vol = vol[:vol.index("/")]

    # fetch year
    try:
        yr = wd.find_element_by_xpath("//h3[@class='js-issue-status text-s']").get_attribute("innerText").split(" ")[-1].replace(")", "").replace("(", "").replace(" ", "")
    except NoSuchElementException:
        continue
    
    print("Reading Volume from Year, " + yr + " : "  + volurl)
    
    # fetch article urls
    articleUrls = []
    a_tags = wd.find_elements_by_xpath("//a[@href]")
    for tag in a_tags:
        if (("article/pii" in tag.get_attribute("href")) and (not "pdf" in tag.get_attribute("href")) and (not "Editorial Board" in tag.text) and (not tag.get_attribute("href") in articleUrls)):
            articleUrls.append(tag.get_attribute("href"))

    for articleurl in articleUrls:
        # access article page
        wd.get(articleurl)
        time.sleep(0.8)
        #print("Reading article, " + articleurl)

        # fetch authors
        authors = []
        auth_gn_tags = wd.find_elements_by_xpath("//span[@class='text given-name']")
        auth_sn_tags = wd.find_elements_by_xpath("//span[@class='text surname']")
        for j in range(len(auth_gn_tags)):
            authors.append(auth_gn_tags[j].text + " " + auth_sn_tags[j].text)
        if(len(authors) == 0):
            continue
        #print(authors)

        # fetch title
        try:
            title = wd.find_element_by_class_name("title-text").text
        except NoSuchElementException:
            continue
        #print(title)

        # fetch keywords
        keywords = []
        kwd_header = wd.find_elements_by_class_name("keywords-section")
        for candid in kwd_header:
            try:
                if(candid.find_element_by_xpath("./h2").text == "Keywords"):
                    kwd_tags = candid.find_elements_by_xpath("./div")
                    for kwd in kwd_tags:
                        keywords.append(kwd.text)
                else:
                    continue
            except NoSuchElementException:
                continue
        while '' in keywords:
            keywords.remove('')
        if(len(keywords) == 0):
            continue
        #print(keywords)

        # write data to excel file
        print("Writing, " + title)
        ws = wb["kwd"]
        ws.cell(row = r, column = 1).value = title
        ws.cell(row = r, column = 2).value = "Information Economics and Policy"
        ws.cell(row = r, column = 3).value = yr
        for j in range(len(keywords)):
            ws.cell(row = r, column = 4 + j).value = keywords[j]
        ws = wb["auth"]
        ws.cell(row = r, column = 1).value = title
        for j in range(len(authors)):
            ws.cell(row = r, column = 2 + j).value = authors[j]
        r += 1

    # backup for every twenty volumes
    if(volUrls.index(volurl) % 20 == 0):
        print("Saving backup file : " + str(volUrls.index(volurl) + 1) + " out of " + str(len(volUrls)))
        wb.save(filename = TARGETPATH + "information_economics_and_policy_backup.xlsx")
        time.sleep(20)
        #files.download(TARGETPATH + "information_economics_and_policy_backup.xlsx")

print("Saving file")
wb.save(filename = TARGETPATH + "information_economics_and_policy.xlsx")

Reading Volume from Year, 2020 : https://www.sciencedirect.com/journal/information-economics-and-policy/vol/50/suppl/C
Writing, Freemium competition among ad-sponsored platforms
Writing, Marketplace or reselling? A signalling model
Writing, Institutions and Telecommunications Investment
Writing, Information advantage and minimum wage
Saving backup file : 1 out of 121
Reading Volume from Year, 2019 : https://www.sciencedirect.com/journal/information-economics-and-policy/vol/49/suppl/C
Writing, Social networks and the demand for news
Writing, Thank you for being a friend: The roles of strong and weak social network ties in attracting backers to crowdfunded campaigns
Writing, Upstream regulation, factor demand and productivity: Cross-industry differences in OECD countries, 1975–2007
Reading Volume from Year, 2019 : https://www.sciencedirect.com/journal/information-economics-and-policy/vol/48/suppl/C
Writing, Openness as a business strategy: Historical perspectives on openness in computing

#**Save File**

In [0]:
wb.save(filename = TARGETPATH + "information_economics_and_policy.xlsx")

#**Download FIle**

In [0]:
files.download(TARGETPATH + "information_economics_and_policy.xlsx")